# Размещение баннеров

Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.



Часть открытых данных есть, например, на сайте archive.org:

https://archive.org/details/201309_foursquare_dataset_umn

Скачаем любым удобным образом архив fsq.zip с этой страницы.

Нас будет интересовать файл checkins.dat:



In [5]:
import pandas as pd
import numpy as np
from sklearn.cluster import MeanShift
from scipy.spatial.distance import euclidean

In [6]:
data = pd.read_table('checkins.dat', delimiter='\s*\|\s*', skiprows=[1])

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


In [7]:
data.shape

(1021966, 6)

Для удобной работы с этим документом преобразуем его к формату csv, удалив строки, не содержащие координат — они неинформативны для нас:

In [8]:
data = data.dropna(subset=['latitude', 'longitude'])
data=data[['latitude', 'longitude']]

С помощью pandas построим DataFrame и убедимся, что все 396634 строки с координатами считаны успешно.

In [9]:
print("shape={}".format(data.shape))
data.head()

shape=(396634, 2)


,latitude,longitude
1,38.895112,-77.036366
3,33.800745,-84.410520
7,45.523452,-122.676207
9,40.764462,-111.904565
10,33.448377,-112.074037


## MeanShift clusterization

Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача — хороший повод познакомиться с алгоритмом MeanShift, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в [sklearn user guide](http://scikit-learn.org/stable/modules/clustering.html#mean-shift), а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

In [10]:
ms = MeanShift(bandwidth=0.1)

Примечание: на 396634 строках кластеризация будет работать долго. Быть очень терпеливым не возбраняется — результат от этого только улучшится. Но для того, чтобы сдать задание, понадобится сабсет из первых 100 тысяч строк. Это компромисс между качеством и затраченным временем. Обучение алгоритма на всём датасете занимает около часа, а на 100 тыс. строк — примерно 2 минуты, однако этого достаточно для получения корректных результатов.

In [11]:
data = data[:100000]

In [12]:
ms.fit(data)

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=1, seeds=None)

In [13]:
print("Total amount of unique clusters : {}".format(len(np.unique(ms.labels_))))

Total amount of unique clusters : 3231


Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

In [14]:
labels = np.unique(ms.labels_)
labels = labels[np.bincount(ms.labels_) > 15]

In [15]:
labels.shape

(592,)

При желании увидеть получившиеся результаты на карте можно передать центры получившихся кластеров в один из инструментов визуализации. Например, сайт [mapcustomizer.com](mapcustomizer.com) имеет функцию Bulk Entry, куда можно вставить центры полученных кластеров в формате:

In [16]:
centers = ms.cluster_centers_[labels]

In [17]:
np.savetxt('centers.txt', centers)

## Closest centers

Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу Carnival Cruise Line адреса офисов:

33.751277, -118.188740 (Los Angeles)

25.867736, -80.324116 (Miami)

51.503016, -0.075479 (London)

52.378894, 4.885084 (Amsterdam)

39.366487, 117.036146 (Beijing)

-33.868457, 151.205134 (Sydney)

In [18]:
offices = np.array([[33.751277, -118.188740], [25.867736, -80.324116], [51.503016, -0.075479], [52.378894, 4.885084], 
           [39.366487, 117.036146], [-33.868457, 151.205134]])

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

Примечание: при подсчете расстояний и в кластеризации можно пренебречь тем, что Земля круглая, так как в точках, расположенных близко друг к другу погрешность мала, а в остальных точках значение достаточно велико.

In [29]:
office_distance = lambda center, offices: min([np.linalg.norm(center - office) for office in offices])


def shift(a, pos):
    tmp = a[-1]
    for i in range(len(a) - 1, pos, -1):
        a[i] = a[i - 1]
    if len(a) < 20:
        a.append(tmp)
        

def find_min(centers, offices):
    nearest = [office_distance(centers[0], offices)]
    nearest_centers = [centers[0]]
    for center in centers:
        distance = office_distance(center, offices)
        for pos, ndistance in enumerate(nearest):
            if ndistance > distance:
                shift(nearest, pos)
                shift(nearest_centers, pos)
                nearest[pos] = distance
                nearest_centers[pos] = center
                break
            if pos == len(nearest) - 1 and len(nearest) < 20:
                nearest.append(distance)
                nearest_centers.append(center)
    return nearest, nearest_centers


nearest, nearest_centers = find_min(centers, offices)


In [36]:
nearest

[0.007834758163107856,
 0.0093533161859922255,
 0.022674066158385495,
 0.050058294822787869,
 0.070847732427199731,
 0.13410903336184654,
 0.16740596425035326,
 0.18887596060185083,
 0.19577945647763628,
 0.21181053682436798,
 0.22223329073179071,
 0.27130075950667348,
 0.29497888680045692,
 0.30227011869246051,
 0.30473050307840693,
 0.31488379033627317,
 0.33881047025113181,
 0.34084565332205718,
 0.37868750125029754,
 0.38670622484272771]

## Answer

Для сдачи задания выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса. Ответ в этом задании — широта и долгота этого центра, записанные через пробел.

In [39]:
def write_answer(center):
    with open("banners_answer.txt", "w") as fout:
        fout.write("{} {}".format(center[0], center[1]))
write_answer(nearest_centers[0])